In [ ]:
%%capture
import os
from pathlib import Path

from dj_notebook import activate

env_file = os.environ["META_ENV"]
reports_folder = Path(os.environ["META_REPORTS_FOLDER"])
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
pharmacy_folder = Path(os.environ["META_PHARMACY_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
from edc_pharmacy.models import StockRequestItem, Stock, StockAdjustment
from django_pandas.io import read_frame


In [ ]:
Stock.objects.all().order_by("created").last()

In [ ]:

df = read_frame(StockRequestItem.objects.filter(stock_request__request_identifier="000122"))

In [ ]:
df.query("in_stock==False")

In [ ]:
StockRequestItem.objects.filter(received=True)

In [ ]:
df = read_frame(Stock.objects.filter(from_stock__code="UGNXMR"))


In [ ]:
df[df["unit_qty_in"]==0.0]

In [ ]:
df["unit_qty_in"].dtype

In [ ]:
obj = Stock.objects.get(code="UGNXMR")

In [ ]:
obj.__dict__

In [ ]:
obj.unit_qty_in

In [ ]:
obj.unit_qty_out


In [ ]:
for obj in Stock.objects.filter(container__container_type__name="bucket"):
    obj.save()
    stock_code = obj.code
    stock_unit_qty_in = obj.unit_qty_in
    stock_unit_qty_out = obj.unit_qty_out
    from_stock_count = Stock.objects.filter(from_stock__code=obj.code).count() * 128
    
    
    if not stock_unit_qty_in >= stock_unit_qty_out:
        print(stock_code, stock_unit_qty_in, stock_unit_qty_out, from_stock_count)
        print(f"**Error stock_code={stock_code}")

In [ ]:
obj

In [ ]:
obj_adjustment = StockAdjustment.objects.create(stock=obj, unit_qty_in_old=obj.unit_qty_in, unit_qty_in_new=24576.00)

In [ ]:
obj.refresh_from_db()

In [ ]:
obj.unit_qty_in

In [ ]:
StockAdjustment.objects.all().delete()

In [ ]:
obj_adjustment

In [ ]:
obj_adjustment.__dict__